##==================================================================##
##        File: Electrochemical-plots.ipynb
##      Author: GOTTFRID OLSSON 
##     Created: 2024-04-24
##     Updated: 2024-04-24
##       About: Reads .txt files of electrochemical cycling of cells
##              and plots the data for the Master's Thesis [TIFX05].
##==================================================================##

In [ ]:
# LIBRARIES #

import numpy as np
import CSV_handler as CSV
import plot_functions as f
import matplotlib.pyplot as plt


In [ ]:
# PLOT SETTINGS #

fig_width_cm  = 16
fig_height_cm = 9

font_size_axis   = 13
font_size_tick   = 11
font_size_legend = 9

grid_major = True
grid_minor = False
legend_on = True

f.set_LaTeX_and_CMU(True) #must run before plotting

In [ ]:
# Paths for data

# SEIDEP: Deposition of Li after a 0.010 V hold and reaching a certain value of i_SEI / i_deposition (= \sigma)
SEIDEP_root_path = 'C:\\MASTER-THESIS\\EXPERIMENTAL\\Data\\Battery-cycling\\SEIDEP\\'

# sigma_x, _0x, _00x means: sigma = x, 0.x, or 0.0x
SEIDEP_sigma_1_path_MB   = SEIDEP_root_path + '2024-04-18_SEIDEP-1_T-cell_Li-Cu_1-mm-PP-spacer_75muL-4MLiFSI-DME_inside-glovebox_02_MB_C10.txt' # SEI-formation
SEIDEP_sigma_1_path_CP   = SEIDEP_root_path + '2024-04-18_SEIDEP-1_T-cell_Li-Cu_1-mm-PP-spacer_75muL-4MLiFSI-DME_inside-glovebox_03_CP_C10.txt' # Li-deposition
SEIDEP_sigma_01_path_MB  = SEIDEP_root_path + '2024-04-19_SEIDEP-4_T-cell_Li-Cu_1-mm-PP-spacer_75muL-4MLiFSI-DME_inside-glovebox_02_MB_C10.txt'
SEIDEP_sigma_01_path_CP  = SEIDEP_root_path + '2024-04-19_SEIDEP-4_T-cell_Li-Cu_1-mm-PP-spacer_75muL-4MLiFSI-DME_inside-glovebox_03_CP_C10.txt'
SEIDEP_sigma_005_path_MB = SEIDEP_root_path + '2024-04-22_SEIDEP-5_T-cell_Li-Cu_1-mm-PP-spacer_75muL-4MLiFSI-DME_inside-glovebox_02_MB_C05.txt'
SEIDEP_sigma_005_path_CP = SEIDEP_root_path + '2024-04-22_SEIDEP-5_T-cell_Li-Cu_1-mm-PP-spacer_75muL-4MLiFSI-DME_inside-glovebox_03_CP_C05.txt'

SEIDEP_sigma_1_MB_df   = CSV.read(SEIDEP_sigma_1_path_MB) # time/s, potential/V, current/mA  (potential = potential of working electrode vs. reference elctrode)
SEIDEP_sigma_01_MB_df  = CSV.read(SEIDEP_sigma_01_path_MB)
SEIDEP_sigma_005_MB_df = CSV.read(SEIDEP_sigma_005_path_MB)

print(SEIDEP_sigma_1_MB_df)
print(SEIDEP_sigma_01_MB_df)
print(SEIDEP_sigma_005_MB_df)

In [ ]:
# PLOT ... #

# Create figure on which axis lives, one axis per subplot
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(fig_width_cm/2.54, fig_height_cm/2.54), sharex=False, sharey=False)

# Plot your data (axs.plot, .errorbar, .hist, ...)
axs.plot(x_data, y_data, linewidth=1.5, linestyle='', color='k', marker='o', markersize='4.5', label='Measured data')
#axs.plot(fit_x, fit_y, color='k', linestyle='-', label='Linear fit')

# Settings for each axis (axs)
f.set_font_size(axis=font_size_axis, tick=font_size_tick, legend=font_size_legend)
f.set_axis_scale(   axs, xScale_string='linear', yScale_string='linear')
f.set_axis_labels(  axs, x_label=x_label, y_label=y_label)
f.set_axis_invert(  axs, x_invert=False, y_invert=False)
f.set_axis_limits(  axs, xmin=x_lim[0], xmax=x_lim[1], ymin=y_lim[0], ymax=y_lim[1])
f.set_grid(         axs, grid_major_on=grid_major, grid_major_linewidth=0.7, grid_minor_on=grid_minor, grid_minor_linewidth=0.3) # set_grid must be after set_axis_scale for some reason (at least with 'log')
f.set_legend(       axs, legend_on=legend_on, alpha=1.0, location='best')

f.align_labels(fig)
f.set_layout_tight(fig)
#f.export_figure_as_pdf(PDF_path)

plt.show()